# Control any Robotic Arm with ROS
 If you wanted to develop a robot, you had to build a complete system: a physical device, of course, but also the control systems, interface, and inspection tools required to get the robot up and running as a test platform.

We have to:
- Create the Robot arm model
- Import the Robot arm model in ROS
- Control the kinematics 

### Install some packages

First of all we need to install:
- urdf tutorial package
- the moveit package.

The simplest way to install MoveIt is from pre-built binaries (Debian):

In [ ]:
rosdep update
sudo apt-get update
sudo apt-get install python3
sudo apt-get install ros-melodic-urdf-tutorial
sudo apt install ros-melodic-moveit

#### Documentation
Interesting links are:
- ROS Tutorial: Create an arm on a mobile robot using Moveit!: https://www.youtube.com/watch?v=l4dtSRvlAjg
- Control any Robotic Arm with ROS: https://blog.usejournal.com/control-any-robotic-arm-with-ros-b10a3115306c
- Udemy course: https://www.udemy.com/course/robotics-with-ros-build-robotic-arm-in-gazebo-and-moveit/
- Udemy course repository: https://github.com/noshluk2/4_Robotic-Arm-Manupilation-using-Gazbo-and-Moveit
- How to control a robot arm with the Arduino and ROS: https://maker.pro/arduino/tutorial/how-to-control-a-robot-arm-with-ros-and-arduino
- Moveit tutorials: http://docs.ros.org/en/melodic/api/moveit_tutorials/html/index.html
- Moveit Setup Assistant Tutorial: http://docs.ros.org/en/melodic/api/moveit_tutorials/html/doc/setup_assistant/setup_assistant_tutorial.html

Public repositories in:
- https://bitbucket.org/theconstructcore/
- https://bitbucket.org/theconstructcore/my-robotic-manipulator/src/master/
- https://bitbucket.org/theconstructcore/two-wheeled-robot-motion-planning/src/master/


## 1.Create the Robot arm model

First of all, we create a "arm_kinematics_ws" where we will install the packages:
- robot_description package: where the URDF model of robotic arm is located. This URDF file is created from scratch or obtained from a SolidWorks plugin
- robot kinematics control package: the moveit_config package

Create a new repository "arm_kinematics_ws" in your github.

Syncronise the repository in your local computer.

Create the src folder and compile the workspace.

We can create an arm model in three different ways:
 - From scratch
 - From SolidWorks
 - From an existing package in ROS

## Design a 3D simple robot arm model from scratch

ROS and MoveIt require our robotic arm model files to be in URDF. The Unified Robotic Description Format (URDF) is an XML file format used in ROS to describe all elements of a robot. It lists all the kinematic properties of the links and joints in a chain from base to tip. And we are going to visualize each step on RViz. ROS also accepts .xacro files which are very similar to urdf and are easier to write.

We can make use of any robot arm design files available on the internet in CAD neutral format (such as STL). Alternatively, one can design it from scratch using any of the popular CAD software packages such as SolidWorks.

For any robotic arm we will have a number of frames that are attach to a robotic arm. After defining the joints, we need to known an important concept, Parent and Child. The different joints are connected between links. The first joint will be the Parent, and the second joint that is connected to this one is the Child. The one which gives access to the origin is the Parent. 

First of all you need to create a new package for robot description. The package is called "custom_robotic_arm". 

In [ ]:
catkin_create_pkg custom_robotic_arm rospy std_msgs

Using Visual Studio Code, we are going to create "urdf" and "launch" folders inside the new created package. 
In the "urdf" folder, we create a file named "simple_arm.urdf", where we are going to describe all the elements of the robot. Now, we program it and we start with the version of excelence and defining the name of the robot. Below, is the code with the things mentioned above and one fixed link and its joint, and also another one with a continuous joint. 



In [ ]:
<!-- Defining the version and the name of the robot -->
<?xml version="1.0"?>
<robot name="luqman_bot">
<!-- First connection to the base link-->
  <link name="world"/>
    <link name="base_link">
        <visual>
        <!-- Geometry and material of the baselink -->
            <geometry>
                <cylinder length="0.05" radius="0.1"/>
            </geometry>
            <material name="silver">
                <color rgba="0.75 0.75 0.75 1"/>
            </material>
            <!-- Location of the base link -->
            <origin rpy="0 0 0" xyz="0 0 0.025"/>
        </visual>
    <!-- Definition of the collision properties -->
    <collision>
        <geometry>
            <cylinder length="0.05" radius="0.1"/>
        </geometry>
            <origin rpy="0 0 0" xyz="0 0 0.025"/>
    </collision>
    <!-- Definition of the intertial properties -->
    <!-- It defines the properties that will act if there is a collision -->
    <inertial>
        <mass value="1.0"/>
        <origin rpy="0 0 0" xyz="0 0 0.025"/>
        <inertia ixx="0.0027" iyy="0.0027" izz="0.005" ixy="0" ixz="0" iyz="0"/>
    </inertial>
    </link>
<!-- Definition of the joint for the link created -->
<!-- This is a fixed link, it is not going to move -->
<!-- Defining the connection between the world and the base link, with a fixed joint -->
  <joint name="fixed" type="fixed">
    <parent link="world"/>
    <child link="base_link"/>
  </joint>

<!-- New link connected to the base one, with a joing that will move -->
<!-- Definition of the geometry and the mechanical properties -->
  <link name="One">
    <visual>
        <geometry>
            <cylinder length="0.5" radius="0.05"/>
        </geometry>
        <material name="silver">
            <color rgba="0.75 0.75 0.75 1"/>
        </material>
        <origin rpy="0 0 0" xyz="0 0 0.25"/>
    </visual>
    <collision>
        <geometry>
            <cylinder length="0.5" radius="0.05"/>
        </geometry>
        <origin rpy="0 0 0" xyz="0 0 0.25"/>
    </collision>
    <inertial>
        <mass value="1.0"/>
        <origin rpy="0 0 0" xyz="0 0 0.25"/>
        <inertia ixx="0.02146" iyy="0.02146" izz="0.00125"
        ixy="0" ixz="0" iyz="0"/>
    </inertial>
  </link>
  <!-- Defining a continuous joint, which can rotate  -->
  <!-- In this case, we have to define the axis and origin -->
  <joint name="One_joint" type="continuous">
    <axis xyz="0 0 1"/>
    <parent link="base_link"/>
    <child link="One"/>
    <origin rpy="0 0 0" xyz="0.0 0.0 0.05"/>
  </joint>

We can repeat this step, as many times as links and joints we want our robot to have. In this case, as it is a simple arm, it is going to have four joints and links. 
To observe the evolution of the robot, we use RViz with the following command on the terminal.  

In [ ]:
roslaunch urdf_tutorial display.launch model:='$(find custom_robotic_arm)/urdf/simple_arm.urdf'

### Controller in Gazebo
Once we have created the robotic arm model, we are going to build a controller to use it in Gazebo environment. On this step we are going to use, Joint State Publisher, that will publish the joints so the simulation is easier and we know the current position of the robot. And Gazebo Controller plugin, to control the robot movements. 

First of all, we need to create a launch file named "arm_gazebo.launch", that contains different nodes from different packages. We want this file to launch de Gazebo world and show our 3D robot model. 

In [ ]:
<launch>
<!-- First we reload our model with an empty world on Gazebo -->
    <param name="robot_description" textfile="$(find custom_robotic_arm)/urdf/simple_arm.urdf"/>
    <include file="$(find gazebo_ros)/launch/empty_world.launch" />
    <node name="spawn_urdf" pkg="gazebo_ros" type="spawn_model" args="-param robot_description -urdf -model luqman_bot"    />
    <rosparam file ="$(find custom_robotic_arm)/config/controller.yaml"/>
    <node name= "controller_spawner" pkg= "controller_manager" type="spawner" args="
    arm_controller"/>
    <node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher"/> 
</launch>

However, we are going to have some problems. Any robot in Gazebo needs a proper control transmission. So, we need to create transmissions and actuators in the joints of the robot. We go back to the "simple_arm.urdf" file, and we will define the transmission and actuators for each joint.
Here it is an example of a transmission for one joint. 

In [ ]:
<!-- We define the simple transmissions on the first joint -->
<transmission name ="Trans_1">
    <type>transmission_interface/SimpleTransmission</type>
    <joint name="One_joint">
        <hardwareInterface>PositionJointInterface</hardwareInterface>
    </joint>
    <!-- Defining the actuator on the first joint, using position interface-->
    <actuator name="Motor1">
        <hardwareInterface>PositionJointInterface</hardwareInterface>
        <mechanicalReduction>1</mechanicalReduction>
    </actuator>
</transmission>

For any robot we need a controller, so we need a Gazebo plugin that controls the transmission and actuations, and another one that publish states, defining all the joints. We come back to "simple_arm.urdf", and we write the following command. 

In [ ]:
<gazebo>
<!-- Defining control package of Gazebo -->
    <plugin name="control" filename="libgazebo_ros_control.so"/>
</gazebo>
<gazebo>
<!-- Plugin that publish states -->
<plugin name="joint_state_publisher" filename="libgazebo_ros_joint_state_publisher.so">
        <jointName>One_joint, Two_joint, Three_joint, Four_joint</jointName>
    </plugin>
</gazebo>

After that we need to create a controller. We will name the file "controller.yaml" and save it in a new folder named "config". This controller is a joint trajectory controller.

In [ ]:
arm_controller:
  type: "position_controllers/JointTrajectoryController"
  joints:
    - One_joint
    - Two_joint
    - Three_joint
    - Four_joint

To observe if the robot looks fine, we will write the following command:

In [ ]:
roslaunch custom_robotic_arm arm_gazebo.launch

If we call "rostopic list", we will see a controller which provides as to control the simulation in Gazebo. 

(After applying the next command, run "roscore" to avoid some problems)

On the topic list there is a arm controller command that will move the robot simulation. 

In [ ]:
rostopic pub /arm_controller/command trajectory_msgs/JointTrajectory '{joint_names:["One_joint","Two_joint","Three_joint,"Four_joint"],points:[{positions:[0.1,-0.5,0.5,0.75],time_from_start:[1,0] }]}'

Depending on the values of positions, the movement will be different. 

In Gazebo we have seen real world simulation, in RViz we see what a robot sees. Thanks to the plugin of joint_state_publisher, that publish all the current state of the joints. First, we need to open rviz. Then on the 'Displays' bar, select "base_link" on 'Fixed Frame'. After that, click on the botton 'Add', and select 'RobotModel' (visualize the robot) and TF (to see the current state of each joint). 

### Controller for Real World Applications with Move-it

To start the MoveIt Setup Assistant:

In [ ]:
roslaunch moveit_setup_assistant setup_assistant.launch


Follow the "Moveit Setup Assistant" Tutorial to create the moveit_config package for the panda robot arm

http://docs.ros.org/en/melodic/api/moveit_tutorials/html/doc/setup_assistant/setup_assistant_tutorial.html

You have now the panda_moveit_config package properly installed

Carefully:
- Choose the robot model created on the previous sections, "simple_arm.urdf".
- Generate Collision Matrix
- Add the links and joints "Planning groups"
- Generate a new folder named "moveit_arm", next to the custom_robotic_arm folder. 



In [ ]:
roslaunch moveit_arm demo.launch

You can make the robot do some random movements. On the section 'MotionPlanning', in the tab 'Planning', you can select a random goal state. You 'Plan' the movement, and then 'Execute'. Then, you see the robot start to move. 

Now we are going to make a synchronization between Moveit and Gazebo. Get that when we plan a movement in the robot, it does the same in the real world. 

We create a file "controller.yaml" on the 'config' folder. That will we the controller that we built in Gazebo configuration and that MoveIt can use it. 

In [ ]:
controller_manager_ns: /
controller_list:
  - name: arm_controller
    action_ns: follow_joint_trajectory
    type: FollowJointTrajectory
    joints:
      - One_joint
      - Two_joint
      - Three_joint
      - Four_joint

Now, we need to change the file "luqman_bot_moveit_controller_manager.launch.xml". It is going to use the controller in Gazebo and link it with the Moveit one. Thanks to that, when we move the robot in Moveit, it will move in Gazebo also. 

In [ ]:
<launch>
    <param name ="moveit_controller_manager"
    value="moveit_simple_controller_manager/MoveItSimpleControllerManager"/>
    <param name="controller_manager_name" value="/" />
    <param name="use_controller_manager" value="true" />
  
    <rosparam file="$(find moveit_arm)/config/controller.yaml" />

</launch>

FInally, we create a new file named "Final.launch". That control the robot on both simulations programs. 

In [ ]:
<launch>
    <include file="$(find custom_robotic_arm)/launch/arm_gazebo.launch" />
    <include file="$(find moveit_arm)/launch/move_group.launch" />
    <include file="$(find moveit_arm)/launch/moveit_rviz.launch" >
        <arg name="config" value="True" />
    </include>
</launch>

When we run it, both programs will open, and when we plan some movement in MoveIt, it will be repeated on Gazebo. 

In [ ]:
roslaunch moveit_arm Final.launch

## Generate URDF model file from SolidWorks
Fortunately, due to a few talented ROS developers, there exists a convenient plugin in Solidworks to export our assembly in URDF format.

Assuming you have your model completely set up as a SolidWorks assembly, we now use the SolidWorks to URDF Exporter: http://wiki.ros.org/sw_urdf_exporter

Follow the tutorial to obtain the URDF file: https://blog.usejournal.com/control-any-robotic-arm-with-ros-b10a3115306c

The built package will contain directories for meshes, textures and robots. It will also contain a ROS package.xml (manifest) file so you can use this as a ROS package by just copying it to your ROS system. The path locations in the URDF are relative to the package itself. Use proper ROS package names to avoid errors.

## Import the Robot arm model in ROS

## Install robot_description package

For this tutorial, make sure you have the Franka description package for Melodic:

In [ ]:
sudo apt-get install ros-melodic-franka-description

Copy the folder "franka_description" to src folder

Compile again the workspace

Verify the lasts lines in .bashrc file

In [ ]:
source /opt/ros/melodic/setup.bash
source /media/sf_github_manelpuig/arm_kinematics_ws/devel/setup.bash

## Create a moveit_config package

To start the MoveIt Setup Assistant:

In [ ]:
roslaunch moveit_setup_assistant setup_assistant.launch

Follow the "Moveit Setup Assistant" Tutorial to create the moveit_config package for the panda robot arm

http://docs.ros.org/en/melodic/api/moveit_tutorials/html/doc/setup_assistant/setup_assistant_tutorial.html

You have now the panda_moveit_config package properly installed

Carefully:
- add the links and joints "Planning groups"
- add the urdf file generated in the robots folder
You need some modifications to this package:
- add the created urdf file in robots folder
- change the gazebo.launch file

## Robot arm Kinematics control 
The quickest way to get started using MoveIt is through its RViz plugin. Rviz is the primary visualizer in ROS and an incredibly useful tool for debugging robotics.

http://docs.ros.org/en/melodic/api/moveit_tutorials/html/doc/quickstart_in_rviz/quickstart_in_rviz_tutorial.html

In [ ]:
roslaunch panda_moveit_config demo.launch rviz_tutorial:=true

Some important configuration parameters:
- In Context Motion Planning Menu: Planning Library OMPL select RTTConnect
- In Planning Request: Interactive marker size to 0,3
- Select "Allow External Communication to use a joystick

<img src="./Images/1_panda_kine_rviz1.png">

In [ ]:
roslaunch gazebo_ros empty_world.launch paused:=true use_sim_time:=false gui:=true throttled:=false recording:=false debug:=true

In [ ]:
rosrun gazebo_ros spawn_model -file /media/sf_github_manelpuig/arm_kinematics_ws/src/franka_description/robots/panda_arm.urdf -urdf -x 0 -y 0 -z 0 -model panda

Change the gazebo.launch file considering:
- urdf path to the urdf file and not the xacro file
- the model has to be changed to panda (not robot as defauld)